## Deploying and Scoring KNIME Polynomial Regression Model

In [1]:
import numpy as np
import pandas as pd

# Features Scaling
from sklearn.preprocessing import MinMaxScaler
# Script Serving
from jpmml_evaluator import make_evaluator
from jpmml_evaluator.py4j import Py4JBackend
# REST Serving
from openscoring import Openscoring
# Performance Metrics
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

## Loading Data

In [2]:
# Load Data
df = pd.read_csv("../data/california_housing.csv")
df_sample = df.sample(10)

In [3]:
# Sacle Data
mms = MinMaxScaler(feature_range=(0, 1))
np_scale = mms.fit_transform(df_sample)
df_scale = pd.DataFrame(np_scale, columns=df_sample.columns)
df_scale

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,PRICE
0,1.000000,0.585366,0.686250,0.274889,0.126798,0.188338,0.683748,0.058939,1.000000
1,0.148470,0.804878,0.037410,0.375060,0.323336,1.000000,0.210835,0.795678,0.511050
2,0.660123,0.000000,1.000000,1.000000,1.000000,0.616763,0.291362,0.807466,0.461326
3,0.282862,0.585366,0.739437,0.671483,0.007071,0.076515,0.622255,0.493124,0.107735
4,0.000000,0.439024,0.715830,0.621014,0.017557,0.271898,1.000000,0.096267,0.000000
5,0.069692,0.195122,0.000000,0.167054,0.000244,0.364034,0.000000,1.000000,0.403867
6,0.105449,0.926829,0.394389,0.057741,0.009022,0.016021,0.787701,0.170923,0.115470
7,0.677707,1.000000,0.633398,0.182150,0.000000,0.000000,0.761347,0.000000,0.575138
8,0.366380,0.634146,0.301488,0.977267,0.208486,0.626243,0.193265,0.764244,0.747514
9,0.513083,0.707317,0.540186,0.000000,0.301634,0.284320,0.226940,0.848723,0.696685


In [4]:
X = df_scale.drop('PRICE', axis=1)
y = df_scale[['PRICE']]

## Model Script Serving

In [5]:
# Load PMML model
evaluator = make_evaluator(backend=Py4JBackend(), path="../data/ridge-regression-knime.pmml")
# Perform automated QA
evaluator.verify()

In [6]:
# Model Serving
y_pred = evaluator.evaluateAll(X)
y_pred.head(10)

,PRICE
0,0.808301
1,0.352557
2,0.483515
3,0.316132
4,-0.066234
5,0.246622
6,0.182825
7,0.664020
8,0.388177
9,0.478649


In [7]:
# Model Performance
print(f'R-Squared: {np.round(r2_score(y, y_pred), decimals=3)*100}%')
print(f'MSE: {np.round(mean_squared_error(y, y_pred), decimals=3)*100}%')

R-Squared: 64.1%
MSE: 3.2%


## Model REST Serving

In [8]:
# Creating an Openscoring object
# Openscoring server have to be running on base-url
os = Openscoring(base_url = "http://localhost:8080", token="secret")

In [9]:
# Deploying a PMML model:
model_name = 'ridge-regression-knime'
os.deployFile(model_name, "../data/ridge-regression-knime.pmml")

In [10]:
# Model serving with data records from DataFrame
dfResponse = os.evaluateCsv(model_name, X)
print(dfResponse.head(10))

      PRICE
0  0.808301
1  0.352557
2  0.483515
3  0.316132
4 -0.066234
5  0.246622
6  0.182825
7  0.664020
8  0.388177
9  0.478649


In [11]:
# Model Performance
print(f'R-Squared: {np.round(r2_score(y, dfResponse), decimals=3)*100}%')
print(f'MSE: {np.round(mean_squared_error(y, dfResponse), decimals=3)*100}%')

R-Squared: 64.1%
MSE: 3.2%
